In [128]:
import geopandas as gpd
import pandas as pd
import rasterio as rio
from rasterio import transform
from rasterio.features import rasterize
import json

In [116]:
# Open soil map
soil_pedo = gpd.read_file(
    "../data/BRO_Soil_Map_Netherlands/BRO_Soil_Map_Netherlands.gpkg",
    layer="areaofpedologicalinterest",
)

soil_area = gpd.read_file(
    "../data/BRO_Soil_Map_Netherlands/BRO_Soil_Map_Netherlands.gpkg",
    layer="soilarea",
)

soil_unit = gpd.read_file(
    "../data/BRO_Soil_Map_Netherlands/BRO_Soil_Map_Netherlands.gpkg",
    layer="soilarea_soilunit",
)

soil_unit_info = gpd.read_file(
    "../data/BRO_Soil_Map_Netherlands/BRO_Soil_Map_Netherlands.gpkg",
    layer="soil_units",
)

In [117]:
soil_pedo_label = soil_pedo["pedologicalinterest"].unique()
print(";".join(soil_pedo_label))

soil_pedo_label_en = "Heavily raised terrain;Terp;Heavily leveled;Heavily processed terrain;Heavily excavated terrain;Quarry;Water;Swamp;Built-up area;Dike;Mine dump;Upland;Land raised with household waste".split(
    ";"
)

soil_pedo_label_remap = {}
for x in range(len(soil_pedo_label_en)):
    soil_pedo_label_remap[soil_pedo_label[x]] = soil_pedo_label_en[x]

soil_pedo["label_en"] = soil_pedo["pedologicalinterest"]
soil_pedo = soil_pedo.replace({"label_en": soil_pedo_label_remap})
soil_pedo

Sterk opgehoogd terrein;Terp;Sterk ge-egaliseerd;Sterk verwerkt terrein;Sterk afgegraven terrein;Groeve;Water;Moeras;Bebouwd gebied;Dijk;Mijnstort;Bovenland;Met huisvuil opgehoogde gronden


,maparea_id,maparea_collection,beginlifespan,endlifespan,pedologicalinterest,geometry,label_en
0,V2023-1.areaofpedologicalinterrest.0000000027,Eerste opname met uitgebreide veldverkenning,None,None,Sterk opgehoogd terrein,"POLYGON ((59361 387558, 59330 387662, 59300 38...",Heavily raised terrain
1,V2023-1.areaofpedologicalinterrest.0000000028,Eerste opname met uitgebreide veldverkenning,None,None,Sterk opgehoogd terrein,"POLYGON ((59017 388003, 58891 388008, 58875 38...",Heavily raised terrain
2,V2023-1.areaofpedologicalinterrest.0000000022,Actualisatie met uitgebreide veldverkenning 2010,None,None,Sterk opgehoogd terrein,"POLYGON ((256321.176 522624.758, 256306.835 52...",Heavily raised terrain
3,V2023-1.areaofpedologicalinterrest.0000000032,Eerste opname met uitgebreide veldverkenning,None,None,Sterk opgehoogd terrein,"POLYGON ((118393 383729, 118427 383738, 118347...",Heavily raised terrain
4,V2023-1.areaofpedologicalinterrest.0000000033,Eerste opname met uitgebreide veldverkenning,None,None,Sterk opgehoogd terrein,"POLYGON ((108288 396779, 108445 396814, 108464...",Heavily raised terrain
...,...,...,...,...,...,...,...
5877,V2023-1.areaofpedologicalinterrest.0000052287,Actualisatie met uitgebreide veldverkenning 2021,None,None,Sterk opgehoogd terrein,"POLYGON ((159841.528 467325.127, 159855.795 46...",Heavily raised terrain
5878,V2023-1.areaofpedologicalinterrest.0000052619,Actualisatie met digitale bodemkartering 2022,None,None,Sterk opgehoogd terrein,"POLYGON ((135333.542 433538.717, 135329.464 43...",Heavily raised terrain
5879,V2023-1.areaofpedologicalinterrest.0000051998,Revisie met detailbodemkaart 2020,None,None,Sterk opgehoogd terrein,"POLYGON ((147113.441 476279.781, 147111.103 47...",Heavily raised terrain
5880,V2023-1.areaofpedologicalinterrest.0000052520,Actualisatie met digitale bodemkartering 2022,None,None,Bebouwd gebied,"POLYGON ((128222 438343.891, 128316.687 438499...",Built-up area


In [118]:
# Soil pedo score
soil_pedo_label_list = """Heavily raised terrain
Terp
Heavily leveled
Heavily processed terrain
Heavily excavated terrain
Quarry
Water
Swamp
Built-up area
Dike
Mine dump
Upland
Land raised with household waste"""
soil_pedo_label_list = soil_pedo_label_list.split("\n")

soil_pedol_label_score = """2
4
1
1
1
2
7
9
0
5
1
6
1"""
soil_pedol_label_score = list(map(lambda x: int(x), soil_pedol_label_score.split("\n")))

# Remap soil to score
remap_soil_pedo_score = {}
for x in range(len(soil_pedo_label_list)):
    remap_soil_pedo_score[soil_pedo_label_list[x]] = soil_pedol_label_score[x]

# Score
soil_pedo["score"] = soil_pedo["label_en"]
soil_pedo = soil_pedo.replace({"score": remap_soil_pedo_score})
soil_pedo

,maparea_id,maparea_collection,beginlifespan,endlifespan,pedologicalinterest,geometry,label_en,score
0,V2023-1.areaofpedologicalinterrest.0000000027,Eerste opname met uitgebreide veldverkenning,None,None,Sterk opgehoogd terrein,"POLYGON ((59361 387558, 59330 387662, 59300 38...",Heavily raised terrain,2
1,V2023-1.areaofpedologicalinterrest.0000000028,Eerste opname met uitgebreide veldverkenning,None,None,Sterk opgehoogd terrein,"POLYGON ((59017 388003, 58891 388008, 58875 38...",Heavily raised terrain,2
2,V2023-1.areaofpedologicalinterrest.0000000022,Actualisatie met uitgebreide veldverkenning 2010,None,None,Sterk opgehoogd terrein,"POLYGON ((256321.176 522624.758, 256306.835 52...",Heavily raised terrain,2
3,V2023-1.areaofpedologicalinterrest.0000000032,Eerste opname met uitgebreide veldverkenning,None,None,Sterk opgehoogd terrein,"POLYGON ((118393 383729, 118427 383738, 118347...",Heavily raised terrain,2
4,V2023-1.areaofpedologicalinterrest.0000000033,Eerste opname met uitgebreide veldverkenning,None,None,Sterk opgehoogd terrein,"POLYGON ((108288 396779, 108445 396814, 108464...",Heavily raised terrain,2
...,...,...,...,...,...,...,...,...
5877,V2023-1.areaofpedologicalinterrest.0000052287,Actualisatie met uitgebreide veldverkenning 2021,None,None,Sterk opgehoogd terrein,"POLYGON ((159841.528 467325.127, 159855.795 46...",Heavily raised terrain,2
5878,V2023-1.areaofpedologicalinterrest.0000052619,Actualisatie met digitale bodemkartering 2022,None,None,Sterk opgehoogd terrein,"POLYGON ((135333.542 433538.717, 135329.464 43...",Heavily raised terrain,2
5879,V2023-1.areaofpedologicalinterrest.0000051998,Revisie met detailbodemkaart 2020,None,None,Sterk opgehoogd terrein,"POLYGON ((147113.441 476279.781, 147111.103 47...",Heavily raised terrain,2
5880,V2023-1.areaofpedologicalinterrest.0000052520,Actualisatie met digitale bodemkartering 2022,None,None,Bebouwd gebied,"POLYGON ((128222 438343.891, 128316.687 438499...",Built-up area,0


In [119]:
# Soil area
soil_area_unit = soil_area.merge(soil_unit, left_on="maparea_id", right_on="maparea_id")

soil_area_unit = soil_area_unit.merge(
    soil_unit_info, left_on="soilunit_code", right_on="code"
)
soil_area_unit

,maparea_id,maparea_collection,beginlifespan,endlifespan,soilslope,geometry,soilunit_code,soilunit_sequencenumber,code,soilclassification,mainsoilclassification,url
0,V2023-1.soilarea.0000007891,Eerste opname met uitgebreide veldverkenning,None,None,Niet opgenomen,"POLYGON ((250817.437 487812.5, 250841.063 4878...",bEZ23,0,bEZ23,Hoge bruine enkeerdgronden; lemig fijn zand,Dikke eerdgronden,https://legenda-bodemkaart.bodemdata.nl/bodemk...
1,V2023-1.soilarea.0000007024,Actualisatie met digitale bodemkartering 2013,None,None,Niet opgenomen,"POLYGON ((206726.516 536030.313, 206716.062 53...",aVz,0,aVz,"Madeveengronden op zand zonder humuspodzol, be...",Veengronden,https://legenda-bodemkaart.bodemdata.nl/bodemk...
2,V2023-1.soilarea.0000007025,Actualisatie met digitale bodemkartering 2013,None,None,Niet opgenomen,"POLYGON ((206429.234 535641.25, 206418.125 535...",aVz,0,aVz,"Madeveengronden op zand zonder humuspodzol, be...",Veengronden,https://legenda-bodemkaart.bodemdata.nl/bodemk...
3,V2023-1.soilarea.0000007026,Actualisatie met digitale bodemkartering 2013,None,None,Niet opgenomen,"POLYGON ((207894.395 536426.077, 207881.125 53...",aVz,0,aVz,"Madeveengronden op zand zonder humuspodzol, be...",Veengronden,https://legenda-bodemkaart.bodemdata.nl/bodemk...
4,V2023-1.soilarea.0000007800,Eerste opname met uitgebreide veldverkenning,None,None,Niet opgenomen,"POLYGON ((260372.859 484005.406, 260353.734 48...",bEZ23,0,bEZ23,Hoge bruine enkeerdgronden; lemig fijn zand,Dikke eerdgronden,https://legenda-bodemkaart.bodemdata.nl/bodemk...
...,...,...,...,...,...,...,...,...,...,...,...,...
48918,V2023-1.soilarea.0000050836,Actualisatie met uitgebreide veldverkenning 2010,None,None,Niet opgenomen,"POLYGON ((263628.525 518695.221, 263617.186 51...",zWz,0,zWz,Moerige eerdgronden met een zanddek en een moe...,Moerige gronden,https://legenda-bodemkaart.bodemdata.nl/bodemk...
48919,V2023-1.soilarea.0000050837,Actualisatie met digitale bodemkartering 2013,None,None,Niet opgenomen,"POLYGON ((195713 497640, 195799 497739, 195880...",zWz,0,zWz,Moerige eerdgronden met een zanddek en een moe...,Moerige gronden,https://legenda-bodemkaart.bodemdata.nl/bodemk...
48920,V2023-1.soilarea.0000051241,Actualisatie met digitale bodemkartering 2018,None,None,Niet opgenomen,"POLYGON ((165128 478373, 165106 478559, 165093...",Hn21,0,Hn21,Veldpodzolgronden; leemarm en zwak lemig fijn ...,Podzolgronden,https://legenda-bodemkaart.bodemdata.nl/bodemk...
48921,V2023-1.soilarea.0000051242,Actualisatie met digitale bodemkartering 2018,None,None,Niet opgenomen,"POLYGON ((181298.203 507226.375, 181222.266 50...",kVd,0,kVd,"Waardveengronden op bagger, verslagen veen, gy...",Veengronden,https://legenda-bodemkaart.bodemdata.nl/bodemk...


In [120]:
soil_names = soil_area_unit["mainsoilclassification"].unique()
print(";".join(soil_names))

soil_names_en = "Thick peat soils;Peat soils;Podzol soils;Non-calcareous sandy soils;Brick soils;Marine clay soils;Loam soils;River clay soils;Defined associations;Boggy soils;Very old marine deposits;Old river clay soils;Very old fluvial deposits;Unmatured mineral soils;Calcareous sandy soils;Boulder clay soils;Limestone weathering soils"
soil_names_en = soil_names_en.split(";")

remap_soil = {}
for x in range(len(soil_names)):
    remap_soil[soil_names[x]] = soil_names_en[x]

soil_area_unit["classification_en"] = soil_area_unit["mainsoilclassification"]
soil_area_unit = soil_area_unit.replace({"classification_en": remap_soil})
soil_area_unit


Dikke eerdgronden;Veengronden;Podzolgronden;Kalkloze zandgronden;Brikgronden;Zeekleigronden;Leemgronden;Rivierkleigronden;Gedefinieerde associaties;Moerige gronden;Zeer oude mariene afzettingen;Oude rivierkleigronden;Zeer oude fluviatiele afzettingen;Niet-gerijpte minerale gronden;Kalkhoudende zandgronden;Keileemgronden;Kalksteen verweringsgronden


,maparea_id,maparea_collection,beginlifespan,endlifespan,soilslope,geometry,soilunit_code,soilunit_sequencenumber,code,soilclassification,mainsoilclassification,url,classification_en
0,V2023-1.soilarea.0000007891,Eerste opname met uitgebreide veldverkenning,None,None,Niet opgenomen,"POLYGON ((250817.437 487812.5, 250841.063 4878...",bEZ23,0,bEZ23,Hoge bruine enkeerdgronden; lemig fijn zand,Dikke eerdgronden,https://legenda-bodemkaart.bodemdata.nl/bodemk...,Thick peat soils
1,V2023-1.soilarea.0000007024,Actualisatie met digitale bodemkartering 2013,None,None,Niet opgenomen,"POLYGON ((206726.516 536030.313, 206716.062 53...",aVz,0,aVz,"Madeveengronden op zand zonder humuspodzol, be...",Veengronden,https://legenda-bodemkaart.bodemdata.nl/bodemk...,Peat soils
2,V2023-1.soilarea.0000007025,Actualisatie met digitale bodemkartering 2013,None,None,Niet opgenomen,"POLYGON ((206429.234 535641.25, 206418.125 535...",aVz,0,aVz,"Madeveengronden op zand zonder humuspodzol, be...",Veengronden,https://legenda-bodemkaart.bodemdata.nl/bodemk...,Peat soils
3,V2023-1.soilarea.0000007026,Actualisatie met digitale bodemkartering 2013,None,None,Niet opgenomen,"POLYGON ((207894.395 536426.077, 207881.125 53...",aVz,0,aVz,"Madeveengronden op zand zonder humuspodzol, be...",Veengronden,https://legenda-bodemkaart.bodemdata.nl/bodemk...,Peat soils
4,V2023-1.soilarea.0000007800,Eerste opname met uitgebreide veldverkenning,None,None,Niet opgenomen,"POLYGON ((260372.859 484005.406, 260353.734 48...",bEZ23,0,bEZ23,Hoge bruine enkeerdgronden; lemig fijn zand,Dikke eerdgronden,https://legenda-bodemkaart.bodemdata.nl/bodemk...,Thick peat soils
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48918,V2023-1.soilarea.0000050836,Actualisatie met uitgebreide veldverkenning 2010,None,None,Niet opgenomen,"POLYGON ((263628.525 518695.221, 263617.186 51...",zWz,0,zWz,Moerige eerdgronden met een zanddek en een moe...,Moerige gronden,https://legenda-bodemkaart.bodemdata.nl/bodemk...,Boggy soils
48919,V2023-1.soilarea.0000050837,Actualisatie met digitale bodemkartering 2013,None,None,Niet opgenomen,"POLYGON ((195713 497640, 195799 497739, 195880...",zWz,0,zWz,Moerige eerdgronden met een zanddek en een moe...,Moerige gronden,https://legenda-bodemkaart.bodemdata.nl/bodemk...,Boggy soils
48920,V2023-1.soilarea.0000051241,Actualisatie met digitale bodemkartering 2018,None,None,Niet opgenomen,"POLYGON ((165128 478373, 165106 478559, 165093...",Hn21,0,Hn21,Veldpodzolgronden; leemarm en zwak lemig fijn ...,Podzolgronden,https://legenda-bodemkaart.bodemdata.nl/bodemk...,Podzol soils
48921,V2023-1.soilarea.0000051242,Actualisatie met digitale bodemkartering 2018,None,None,Niet opgenomen,"POLYGON ((181298.203 507226.375, 181222.266 50...",kVd,0,kVd,"Waardveengronden op bagger, verslagen veen, gy...",Veengronden,https://legenda-bodemkaart.bodemdata.nl/bodemk...,Peat soils


In [121]:
# Soil classing
label_en_list = """Thick peat soils
Peat soils
Podzol soils
Non-calcareous sandy soils
Brick soils
Marine clay soils
Loam soils
River clay soils
Defined associations
Boggy soils
Very old marine deposits
Old river clay soils
Very old fluvial deposits
Unmatured mineral soils
Calcareous sandy soils
Boulder clay soils
Limestone weathering soils"""
label_en_list = label_en_list.split("\n")

score_list = """9
8
4
5
7
7
8
9
7
9
5
8
6
2
7
5
3"""
score_list = list(map(lambda x: int(x), score_list.split("\n")))


# Remap soil to score
remap_soil_score = {}
for x in range(len(label_en_list)):
    remap_soil_score[label_en_list[x]] = score_list[x]

# Score
soil_area_unit["score"] = soil_area_unit["classification_en"]
soil_area_unit = soil_area_unit.replace({"score": remap_soil_score})
soil_area_unit

,maparea_id,maparea_collection,beginlifespan,endlifespan,soilslope,geometry,soilunit_code,soilunit_sequencenumber,code,soilclassification,mainsoilclassification,url,classification_en,score
0,V2023-1.soilarea.0000007891,Eerste opname met uitgebreide veldverkenning,None,None,Niet opgenomen,"POLYGON ((250817.437 487812.5, 250841.063 4878...",bEZ23,0,bEZ23,Hoge bruine enkeerdgronden; lemig fijn zand,Dikke eerdgronden,https://legenda-bodemkaart.bodemdata.nl/bodemk...,Thick peat soils,9
1,V2023-1.soilarea.0000007024,Actualisatie met digitale bodemkartering 2013,None,None,Niet opgenomen,"POLYGON ((206726.516 536030.313, 206716.062 53...",aVz,0,aVz,"Madeveengronden op zand zonder humuspodzol, be...",Veengronden,https://legenda-bodemkaart.bodemdata.nl/bodemk...,Peat soils,8
2,V2023-1.soilarea.0000007025,Actualisatie met digitale bodemkartering 2013,None,None,Niet opgenomen,"POLYGON ((206429.234 535641.25, 206418.125 535...",aVz,0,aVz,"Madeveengronden op zand zonder humuspodzol, be...",Veengronden,https://legenda-bodemkaart.bodemdata.nl/bodemk...,Peat soils,8
3,V2023-1.soilarea.0000007026,Actualisatie met digitale bodemkartering 2013,None,None,Niet opgenomen,"POLYGON ((207894.395 536426.077, 207881.125 53...",aVz,0,aVz,"Madeveengronden op zand zonder humuspodzol, be...",Veengronden,https://legenda-bodemkaart.bodemdata.nl/bodemk...,Peat soils,8
4,V2023-1.soilarea.0000007800,Eerste opname met uitgebreide veldverkenning,None,None,Niet opgenomen,"POLYGON ((260372.859 484005.406, 260353.734 48...",bEZ23,0,bEZ23,Hoge bruine enkeerdgronden; lemig fijn zand,Dikke eerdgronden,https://legenda-bodemkaart.bodemdata.nl/bodemk...,Thick peat soils,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48918,V2023-1.soilarea.0000050836,Actualisatie met uitgebreide veldverkenning 2010,None,None,Niet opgenomen,"POLYGON ((263628.525 518695.221, 263617.186 51...",zWz,0,zWz,Moerige eerdgronden met een zanddek en een moe...,Moerige gronden,https://legenda-bodemkaart.bodemdata.nl/bodemk...,Boggy soils,9
48919,V2023-1.soilarea.0000050837,Actualisatie met digitale bodemkartering 2013,None,None,Niet opgenomen,"POLYGON ((195713 497640, 195799 497739, 195880...",zWz,0,zWz,Moerige eerdgronden met een zanddek en een moe...,Moerige gronden,https://legenda-bodemkaart.bodemdata.nl/bodemk...,Boggy soils,9
48920,V2023-1.soilarea.0000051241,Actualisatie met digitale bodemkartering 2018,None,None,Niet opgenomen,"POLYGON ((165128 478373, 165106 478559, 165093...",Hn21,0,Hn21,Veldpodzolgronden; leemarm en zwak lemig fijn ...,Podzolgronden,https://legenda-bodemkaart.bodemdata.nl/bodemk...,Podzol soils,4
48921,V2023-1.soilarea.0000051242,Actualisatie met digitale bodemkartering 2018,None,None,Niet opgenomen,"POLYGON ((181298.203 507226.375, 181222.266 50...",kVd,0,kVd,"Waardveengronden op bagger, verslagen veen, gy...",Veengronden,https://legenda-bodemkaart.bodemdata.nl/bodemk...,Peat soils,8


In [122]:
# Combine both soil
soil_combined = pd.concat([soil_area_unit, soil_pedo])
soil_combined

,maparea_id,maparea_collection,beginlifespan,endlifespan,soilslope,geometry,soilunit_code,soilunit_sequencenumber,code,soilclassification,mainsoilclassification,url,classification_en,score,pedologicalinterest,label_en
0,V2023-1.soilarea.0000007891,Eerste opname met uitgebreide veldverkenning,None,None,Niet opgenomen,"POLYGON ((250817.437 487812.5, 250841.063 4878...",bEZ23,0.0,bEZ23,Hoge bruine enkeerdgronden; lemig fijn zand,Dikke eerdgronden,https://legenda-bodemkaart.bodemdata.nl/bodemk...,Thick peat soils,9,NaN,NaN
1,V2023-1.soilarea.0000007024,Actualisatie met digitale bodemkartering 2013,None,None,Niet opgenomen,"POLYGON ((206726.516 536030.313, 206716.062 53...",aVz,0.0,aVz,"Madeveengronden op zand zonder humuspodzol, be...",Veengronden,https://legenda-bodemkaart.bodemdata.nl/bodemk...,Peat soils,8,NaN,NaN
2,V2023-1.soilarea.0000007025,Actualisatie met digitale bodemkartering 2013,None,None,Niet opgenomen,"POLYGON ((206429.234 535641.25, 206418.125 535...",aVz,0.0,aVz,"Madeveengronden op zand zonder humuspodzol, be...",Veengronden,https://legenda-bodemkaart.bodemdata.nl/bodemk...,Peat soils,8,NaN,NaN
3,V2023-1.soilarea.0000007026,Actualisatie met digitale bodemkartering 2013,None,None,Niet opgenomen,"POLYGON ((207894.395 536426.077, 207881.125 53...",aVz,0.0,aVz,"Madeveengronden op zand zonder humuspodzol, be...",Veengronden,https://legenda-bodemkaart.bodemdata.nl/bodemk...,Peat soils,8,NaN,NaN
4,V2023-1.soilarea.0000007800,Eerste opname met uitgebreide veldverkenning,None,None,Niet opgenomen,"POLYGON ((260372.859 484005.406, 260353.734 48...",bEZ23,0.0,bEZ23,Hoge bruine enkeerdgronden; lemig fijn zand,Dikke eerdgronden,https://legenda-bodemkaart.bodemdata.nl/bodemk...,Thick peat soils,9,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5877,V2023-1.areaofpedologicalinterrest.0000052287,Actualisatie met uitgebreide veldverkenning 2021,None,None,NaN,"POLYGON ((159841.528 467325.127, 159855.795 46...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,Sterk opgehoogd terrein,Heavily raised terrain
5878,V2023-1.areaofpedologicalinterrest.0000052619,Actualisatie met digitale bodemkartering 2022,None,None,NaN,"POLYGON ((135333.542 433538.717, 135329.464 43...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,Sterk opgehoogd terrein,Heavily raised terrain
5879,V2023-1.areaofpedologicalinterrest.0000051998,Revisie met detailbodemkaart 2020,None,None,NaN,"POLYGON ((147113.441 476279.781, 147111.103 47...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,Sterk opgehoogd terrein,Heavily raised terrain
5880,V2023-1.areaofpedologicalinterrest.0000052520,Actualisatie met digitale bodemkartering 2022,None,None,NaN,"POLYGON ((128222 438343.891, 128316.687 438499...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Bebouwd gebied,Built-up area


In [123]:
soil_combined.to_file("../output/soil_nl_en_scored.shp")

In [124]:
# Get extent of the vector
extent = soil_combined.total_bounds

# Calculate the the pixels potential
height = extent[3] - extent[1]
width = extent[2] - extent[0]
shape = (int(height / 10), int(width / 10))
shape

# Transform affine
affine_transform = transform.from_bounds(*extent, shape[1], shape[0])
affine_transform

# CRS
crs = soil_combined.crs

In [125]:
# Vector to raster
raster = rasterize(
    [data for data in zip(soil_combined.geometry, soil_combined["score"])],
    out_shape=shape,
    transform=affine_transform,
    nodata=-9999,
    fill=-9999,
    dtype="int16",
)
raster

array([[-9999, -9999, -9999, ..., -9999, -9999, -9999],
       [-9999, -9999, -9999, ..., -9999, -9999, -9999],
       [-9999, -9999, -9999, ..., -9999, -9999, -9999],
       ...,
       [-9999, -9999, -9999, ..., -9999, -9999, -9999],
       [-9999, -9999, -9999, ..., -9999, -9999, -9999],
       [-9999, -9999, -9999, ..., -9999, -9999, -9999]],
      shape=(31225, 26446), dtype=int16)

In [126]:
# Save the raster
with rio.open(
    "../output/soil_nl_raster_score.tif",
    "w",
    "COG",
    count=1,
    width=raster.shape[1],
    height=raster.shape[0],
    nodata=-9999,
    compress="lzw",
    transform=affine_transform,
    crs=crs,
    dtype="int16",
) as source:
    source.write(raster, 1)

CPLE_AppDefinedError: Deleting ../output/soil_nl_raster_score.tif failed: Permission denied

In [129]:
# Make cateogorical soil
soil_categorical = soil_combined.copy()
soil_categorical["classification_en"] = soil_categorical["classification_en"].fillna(
    soil_categorical["label_en"]
)
soil_categorical["classification_id"] = soil_categorical["classification_en"]

# Remap class to id
categorical_en = """Heavily raised terrain
Terp
Heavily leveled
Heavily processed terrain
Heavily excavated terrain
Quarry
Water
Swamp
Built-up area
Dike
Mine dump
Upland
Land raised with household waste
Thick peat soils
Peat soils
Podzol soils
Non-calcareous sandy soils
Brick soils
Marine clay soils
Loam soils
River clay soils
Defined associations
Boggy soils
Very old marine deposits
Old river clay soils
Very old fluvial deposits
Unmatured mineral soils
Calcareous sandy soils
Boulder clay soils
Limestone weathering soils""".split("\n")

# Clsss id
class_ids = """1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30""".split("\n")
class_ids = list(map(lambda x: int(x), class_ids))

remap_class_id = {}
remap_class_ids = {}
for x in range(len(class_ids)):
    remap_class_id[categorical_en[x]] = class_ids[x]
    remap_class_ids[class_ids[x]] = categorical_en[x]

print(json.dumps(remap_class_ids))

soil_categorical = soil_categorical.replace({"classification_id": remap_class_id})
soil_categorical

{"1": "Heavily raised terrain", "2": "Terp", "3": "Heavily leveled", "4": "Heavily processed terrain", "5": "Heavily excavated terrain", "6": "Quarry", "7": "Water", "8": "Swamp", "9": "Built-up area", "10": "Dike", "11": "Mine dump", "12": "Upland", "13": "Land raised with household waste", "14": "Thick peat soils", "15": "Peat soils", "16": "Podzol soils", "17": "Non-calcareous sandy soils", "18": "Brick soils", "19": "Marine clay soils", "20": "Loam soils", "21": "River clay soils", "22": "Defined associations", "23": "Boggy soils", "24": "Very old marine deposits", "25": "Old river clay soils", "26": "Very old fluvial deposits", "27": "Unmatured mineral soils", "28": "Calcareous sandy soils", "29": "Boulder clay soils", "30": "Limestone weathering soils"}


,maparea_id,maparea_collection,beginlifespan,endlifespan,soilslope,geometry,soilunit_code,soilunit_sequencenumber,code,soilclassification,mainsoilclassification,url,classification_en,score,pedologicalinterest,label_en,classification_id
0,V2023-1.soilarea.0000007891,Eerste opname met uitgebreide veldverkenning,None,None,Niet opgenomen,"POLYGON ((250817.437 487812.5, 250841.063 4878...",bEZ23,0.0,bEZ23,Hoge bruine enkeerdgronden; lemig fijn zand,Dikke eerdgronden,https://legenda-bodemkaart.bodemdata.nl/bodemk...,Thick peat soils,9,NaN,NaN,14
1,V2023-1.soilarea.0000007024,Actualisatie met digitale bodemkartering 2013,None,None,Niet opgenomen,"POLYGON ((206726.516 536030.313, 206716.062 53...",aVz,0.0,aVz,"Madeveengronden op zand zonder humuspodzol, be...",Veengronden,https://legenda-bodemkaart.bodemdata.nl/bodemk...,Peat soils,8,NaN,NaN,15
2,V2023-1.soilarea.0000007025,Actualisatie met digitale bodemkartering 2013,None,None,Niet opgenomen,"POLYGON ((206429.234 535641.25, 206418.125 535...",aVz,0.0,aVz,"Madeveengronden op zand zonder humuspodzol, be...",Veengronden,https://legenda-bodemkaart.bodemdata.nl/bodemk...,Peat soils,8,NaN,NaN,15
3,V2023-1.soilarea.0000007026,Actualisatie met digitale bodemkartering 2013,None,None,Niet opgenomen,"POLYGON ((207894.395 536426.077, 207881.125 53...",aVz,0.0,aVz,"Madeveengronden op zand zonder humuspodzol, be...",Veengronden,https://legenda-bodemkaart.bodemdata.nl/bodemk...,Peat soils,8,NaN,NaN,15
4,V2023-1.soilarea.0000007800,Eerste opname met uitgebreide veldverkenning,None,None,Niet opgenomen,"POLYGON ((260372.859 484005.406, 260353.734 48...",bEZ23,0.0,bEZ23,Hoge bruine enkeerdgronden; lemig fijn zand,Dikke eerdgronden,https://legenda-bodemkaart.bodemdata.nl/bodemk...,Thick peat soils,9,NaN,NaN,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5877,V2023-1.areaofpedologicalinterrest.0000052287,Actualisatie met uitgebreide veldverkenning 2021,None,None,NaN,"POLYGON ((159841.528 467325.127, 159855.795 46...",NaN,NaN,NaN,NaN,NaN,NaN,Heavily raised terrain,2,Sterk opgehoogd terrein,Heavily raised terrain,1
5878,V2023-1.areaofpedologicalinterrest.0000052619,Actualisatie met digitale bodemkartering 2022,None,None,NaN,"POLYGON ((135333.542 433538.717, 135329.464 43...",NaN,NaN,NaN,NaN,NaN,NaN,Heavily raised terrain,2,Sterk opgehoogd terrein,Heavily raised terrain,1
5879,V2023-1.areaofpedologicalinterrest.0000051998,Revisie met detailbodemkaart 2020,None,None,NaN,"POLYGON ((147113.441 476279.781, 147111.103 47...",NaN,NaN,NaN,NaN,NaN,NaN,Heavily raised terrain,2,Sterk opgehoogd terrein,Heavily raised terrain,1
5880,V2023-1.areaofpedologicalinterrest.0000052520,Actualisatie met digitale bodemkartering 2022,None,None,NaN,"POLYGON ((128222 438343.891, 128316.687 438499...",NaN,NaN,NaN,NaN,NaN,NaN,Built-up area,0,Bebouwd gebied,Built-up area,9


In [114]:
# Vector to raster
raster_soil_category = rasterize(
    [
        data
        for data in zip(
            soil_categorical.geometry, soil_categorical["classification_id"]
        )
    ],
    out_shape=shape,
    transform=affine_transform,
    nodata=-9999,
    fill=-9999,
    dtype="int16",
)
raster_soil_category

# Save the raster
with rio.open(
    "../output/soil_nl_category_raster.tif",
    "w",
    "COG",
    count=1,
    width=raster_soil_category.shape[1],
    height=raster_soil_category.shape[0],
    nodata=-9999,
    compress="lzw",
    transform=affine_transform,
    crs=crs,
    dtype="int16",
) as source:
    source.write(raster_soil_category, 1)